파생변수 : 원래 있던 변수들을 조합하거나 함수를 적용하여 새로 만들어낸 변수


파생변수는 데이터의 특성을 이용하여 분석 효율을 높이는 것이기 때문에 전체 데이터에 대한 파악이 중요할 뿐만 아니라 해당 비즈니스 도메인에 대한 충분한 이해가 수반되어야 한다

무작정 변수를 가공해서 만드는 것이 아니라 데이터의 특성과 흐름을 충분히 파악한 후 아이디어를 얻어서 만드는 것이 효과적이다

파생변수는 기존의 변수를 활용해서 만들어낸 변수이기 때문에 다중공선성 문제가 발생할 가능성이 높다

그렇기 때문에 파생변수를 만든 다음에는 상관분석을 통해 변수 간의 상관성을 확인해야한다

변수들 조합의 주요 속성만 추출해내는 주성분 분석(PCA) 등을 사용할 수 있다

# 파생 변수 생성 실습

# 1. 패키지 임포트

In [32]:
import math
from sklearn import preprocessing
import datetime
from datetime import timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 2. 데이터 불러오기 및 확인

In [33]:
df = pd.read_csv('/content/scanner_data.csv')

df.head()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88


# 3. 두 개의 변수를 결합하여 파생변수 생성

In [34]:
# 구매 상품 당 가격 칼럼 생성
df['Unit_amount'] = df['Sales_Amount']/df['Quantity']

# 총 구매가격 칼럼 생성
df['All_amount'] = df[['Quantity', 'Sales_Amount']].apply(lambda series: series.prod(), axis=1)

df.tail()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Unit_amount,All_amount
131701,131702,04/07/2016,20203,32900,IEV,FO112,3.0,6.46,2.153333,19.38
131702,131703,04/07/2016,20203,32900,N8U,I36F2,1.0,4.50,4.500000,4.50
131703,131704,04/07/2016,20203,32900,U5F,4X8P4,1.0,5.19,5.190000,5.19
131704,131705,04/07/2016,20203,32900,0H2,ZVTO4,1.0,4.57,4.570000,4.57
131705,131706,04/07/2016,20203,32900,Q4N,QM9BP,1.0,13.68,13.680000,13.68


# 4. 로그, 제곱근, 제곱 변환 파생변수 생성

In [35]:
# 방법1. Sales_Amount 칼럼 로그 적용 (+1)
df['Sales_Amount_log'] = preprocessing.scale(np.log(df['Sales_Amount']+1))

# 방법2. Sales_Amount 칼럼 로그 적용 (+1)
df['Sales_Amount_log2'] = df[['Sales_Amount']].apply(lambda x : np.log(x+1))

# Sales_Amount 칼럼 제곱근 적용 (+1)
df['Sales_Amount_sqrt'] = np.sqrt(df['Sales_Amount']+1)

# Sales_Amount 칼럼 제곱 적용
df['Sales_Amount_pow'] = pow(df[['Sales_Amount']], 2)

df.tail()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Unit_amount,All_amount,Sales_Amount_log,Sales_Amount_log2,Sales_Amount_sqrt,Sales_Amount_pow
131701,131702,04/07/2016,20203,32900,IEV,FO112,3.0,6.46,2.153333,19.38,-0.204581,2.009555,2.731300,41.7316
131702,131703,04/07/2016,20203,32900,N8U,I36F2,1.0,4.50,4.500000,4.50,-0.592329,1.704748,2.345208,20.2500
131703,131704,04/07/2016,20203,32900,U5F,4X8P4,1.0,5.19,5.190000,5.19,-0.441982,1.822935,2.487971,26.9361
131704,131705,04/07/2016,20203,32900,0H2,ZVTO4,1.0,4.57,4.570000,4.57,-0.576240,1.717395,2.360085,20.8849
131705,131706,04/07/2016,20203,32900,Q4N,QM9BP,1.0,13.68,13.680000,13.68,0.656548,2.686486,3.831449,187.1424


로그를 취할 때는 0값은 처리가 되지 않기 때문에 기존 값에 1을 더해주고 로그를 취해주거나 임의의 값으로 변환해 줘야 한다

첫번째 로그 적용은 스케일링을 적용한 후에 로그를 취했고, 두번째 방법은 그대로 로그를 취했다

# 5. 월 합계, 평균 구매금액 변수 생성

In [36]:
# date 칼럼 날짜 형식 변환
df['Date2'] = pd.to_datetime(df['Date'],format='%d/%m/%Y')

# 연도 칼럼 생성
df['Year'] = df['Date2'].dt.year

# 월 칼럼 생성
df['Month'] = df['Date2'].dt.month

# 연월별, 고객별 매출 합계, 평균 칼럼 생성
df_sm = df.groupby(['Year', 'Month', 'Customer_ID'])['Sales_Amount'].agg(['sum','mean']).reset_index()

# 기존 일별 테이블에 평균 테이블 조인
df2 = pd.merge(df, df_sm, how='left')

df2.tail()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Unit_amount,All_amount,Sales_Amount_log,Sales_Amount_log2,Sales_Amount_sqrt,Sales_Amount_pow,Date2,Year,Month,sum,mean
131701,131702,04/07/2016,20203,32900,IEV,FO112,3.0,6.46,2.153333,19.38,-0.204581,2.009555,2.731300,41.7316,2016-07-04,2016,7,61.36,7.67
131702,131703,04/07/2016,20203,32900,N8U,I36F2,1.0,4.50,4.500000,4.50,-0.592329,1.704748,2.345208,20.2500,2016-07-04,2016,7,61.36,7.67
131703,131704,04/07/2016,20203,32900,U5F,4X8P4,1.0,5.19,5.190000,5.19,-0.441982,1.822935,2.487971,26.9361,2016-07-04,2016,7,61.36,7.67
131704,131705,04/07/2016,20203,32900,0H2,ZVTO4,1.0,4.57,4.570000,4.57,-0.576240,1.717395,2.360085,20.8849,2016-07-04,2016,7,61.36,7.67
131705,131706,04/07/2016,20203,32900,Q4N,QM9BP,1.0,13.68,13.680000,13.68,0.656548,2.686486,3.831449,187.1424,2016-07-04,2016,7,61.36,7.67


# 6. 월 단위 변수와 일 단위 변수를 활용한 파생변수 생성

In [37]:
# 월 평균 구매금액 대비 일별 구매금액 차이 변수 생성
df['Sales_Amount_Diff'] = df2['mean'] - df2['Sales_Amount']

# 월 평균 구매금액 대비 일별 구매금액 비율 변수 생성
df2['Sales_Amount_UD'] = df2['Sales_Amount'] / df2['mean']

# 월 총 구매금액 대비 일별 구매금액 비율 변수 생성
df2['Sales_Amount_Rto'] = df2['Sales_Amount'] / df2['sum']

df2.head()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Unit_amount,All_amount,...,Sales_Amount_log2,Sales_Amount_sqrt,Sales_Amount_pow,Date2,Year,Month,sum,mean,Sales_Amount_UD,Sales_Amount_Rto
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13,3.13,3.13,...,1.418277,2.032240,9.7969,2016-01-02,2016,1,27.01,6.752500,0.463532,0.115883
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46,5.46,5.46,...,1.865629,2.541653,29.8116,2016-01-02,2016,1,14.05,3.512500,1.554448,0.388612
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35,6.35,6.35,...,1.994700,2.711088,40.3225,2016-01-02,2016,1,135.16,7.113684,0.892646,0.046981
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59,5.59,5.59,...,1.885553,2.567100,31.2481,2016-01-02,2016,1,244.83,9.793200,0.570804,0.022832
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88,6.88,6.88,...,2.064328,2.807134,47.3344,2016-01-02,2016,1,6.88,6.880000,1.000000,1.000000


# 7. 전월 값을 활용한 파생변수 생성

In [44]:
# 4주 뒤 시점 칼럼 생성
df2['Date2_1_m'] = df2['Date2'] +timedelta(weeks=4)

# 4주 뒤 시점연도 칼럼 생성
df['Year_1_m'] = df2['Date2_1_m'].dt.year

# 4주 뒤 시점월 칼럼 생성
df['Month_1_m'] = df2['Date2_1_m'].dt.month

# 4주 전 구매금액 연월별, 고객별 매출 칼럼 생성
df_Mn_1 = df.groupby(['Year_1_m',
                      'Month_1_m',
                      'Customer_ID'])['Sales_Amount'].agg(['sum', 'mean']).reset_index()

# 조인을 위한 칼럼명 변경
df_Mn_1.rename(columns = {'Year_1_m':'Year',
               'Month_1_m' : 'Month', 'sum' : 'sum_1_m', 'mean':'mean_1_m'}, inplace=True)

df2 = pd.merge(df2, df_Mn_1, how='left')

df2.head()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Unit_amount,All_amount,...,Month,sum,mean,Sales_Amount_UD,Sales_Amount_Rto,Date2_1_m,Year_1_m,Month_1_m,sum_1_m,mean_1_m
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13,3.13,3.13,...,1,27.01,6.752500,0.463532,0.115883,2016-01-30,2016,1,3.13,3.13
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46,5.46,5.46,...,1,14.05,3.512500,1.554448,0.388612,2016-01-30,2016,1,5.46,5.46
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35,6.35,6.35,...,1,135.16,7.113684,0.892646,0.046981,2016-01-30,2016,1,6.35,6.35
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59,5.59,5.59,...,1,244.83,9.793200,0.570804,0.022832,2016-01-30,2016,1,5.59,5.59
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88,6.88,6.88,...,1,6.88,6.880000,1.000000,1.000000,2016-01-30,2016,1,6.88,6.88


기존 시점 칼럼인 Date2를 한달 뒤 시점으로 변환한 뒤에 Sales_Amount의 합계와 평균을 구한 다음 기존 한 달 시점 칼럼을 기준으로 조인을 해줘야 한다

만약 전달의 값이 없는 경우에는 결측값이 생길 수 있기 때문에 결측값은 0으로 처리해주어야 한다

# 8. 전월과의 차이 파생변수 생성

In [45]:
# 전월 대비 구매금액 평균 차이 변수 생성
df2['Mn_diff_1_mean'] = df2['mean'] - df2['mean_1_m']

# 전월 대비 총 구매금액 차이 변수 생성
df2['Mn_diff_1_sum'] = df2['sum'] - df2['sum_1_m']

df2.head()

,Unnamed: 0,Date,Customer_ID,Transaction_ID,SKU_Category,SKU,Quantity,Sales_Amount,Unit_amount,All_amount,...,mean,Sales_Amount_UD,Sales_Amount_Rto,Date2_1_m,Year_1_m,Month_1_m,sum_1_m,mean_1_m,Mn_diff_1_mean,Mn_diff_1_sum
0,1,02/01/2016,2547,1,X52,0EM7L,1.0,3.13,3.13,3.13,...,6.752500,0.463532,0.115883,2016-01-30,2016,1,3.13,3.13,3.622500,23.88
1,2,02/01/2016,822,2,2ML,68BRQ,1.0,5.46,5.46,5.46,...,3.512500,1.554448,0.388612,2016-01-30,2016,1,5.46,5.46,-1.947500,8.59
2,3,02/01/2016,3686,3,0H2,CZUZX,1.0,6.35,6.35,6.35,...,7.113684,0.892646,0.046981,2016-01-30,2016,1,6.35,6.35,0.763684,128.81
3,4,02/01/2016,3719,4,0H2,549KK,1.0,5.59,5.59,5.59,...,9.793200,0.570804,0.022832,2016-01-30,2016,1,5.59,5.59,4.203200,239.24
4,5,02/01/2016,9200,5,0H2,K8EHH,1.0,6.88,6.88,6.88,...,6.880000,1.000000,1.000000,2016-01-30,2016,1,6.88,6.88,0.000000,0.00
